# Cargamos el dataframe de la primera notebook

In [37]:
import pandas as pd
import os

# --- PASO 1: Cargar la data del Notebook 1 ---
input_path = '../data/df_final_procesado.pkl'

if os.path.exists(input_path):
    # Cargamos el DataFrame y lo llamamos 'df' para que coincida con tu código
    df = pd.read_pickle(input_path)
    print(f"✅ Data cargada. Registros: {df.shape[0]}")
else:
    print("❌ Error: No se encuentra el archivo .pkl. Ejecuta el Notebook 1 primero.")

# --- PASO 2: Tu código para crear la matriz ---
# Crear la matriz: Index=Películas, Columns=Usuarios, Values=Ratings
# Rellenamos con 0 los nulos (asumimos que no verla = 0 interés por ahora)
movie_user_matrix = df.pivot_table(index='title', columns='userId', values='rating').fillna(0)

print(f"Forma de la matriz: {movie_user_matrix.shape}")
# Resultado esperado: (9719, 610) aprox

✅ Data cargada. Registros: 100836
Forma de la matriz: (9719, 610)


# 1. Crear la Matriz Utilidad (Pivot Table)
### Necesitamos reorganizar los datos para que las filas sean peliculas y las columnas sean usuarios

In [26]:
# Crear la matriz: Index=Películas, Columns=Usuarios, Values=Ratings
# Rellenamos con 0 los nulos (asumimos que no verla = 0 interés por ahora)
movie_user_matrix = df.pivot_table(index='title', columns='userId', values='rating').fillna(0)

print(f"Forma de la matriz: {movie_user_matrix.shape}")
# Deberías ver algo como (9719, 610) -> 9k películas, 610 usuarios
display(movie_user_matrix.head())

Forma de la matriz: (9719, 610)


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 2. Aplicar SVD (Reducción de Dimensionalidad)
### Usaremos TruncatedSVD de Scikit-Learn. Vamos a comprimir esos 610 usuarios en, digamos, 12 "componentes" principales. Esto elimina el ruido y captura la "esencia" de la película.

In [32]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

# Comprimimos la matriz a 12 componentes latentes
SVD = TruncatedSVD(n_components=12, random_state=42)
matrix_reduced = SVD.fit_transform(movie_user_matrix)

print(f"Forma de la matriz reducida: {matrix_reduced.shape}")
# Ahora tenemos (9719, 12). ¡Mucho más eficiente!

Forma de la matriz reducida: (9719, 12)


# 3. Calcular Correlaciones (El motor de recomendación)
### Ahora que cada película está representada por 12 números (su "ADN"), calculamos qué tan parecidos son esos ADNs entre sí usando la Correlación de Pearson.3. Calcular Correlaciones (El motor de recomendación)

In [33]:
# Calculamos la matriz de correlación de todas las películas contra todas
# Nota: corrcoef de numpy espera filas como variables, que es lo que tenemos
corr_matrix = np.corrcoef(matrix_reduced)

print(f"Forma de la matriz de correlación: {corr_matrix.shape}")
# Debería ser (9719, 9719). Una matriz cuadrada comparando cada peli con las demás.

Forma de la matriz de correlación: (9719, 9719)


# Paso 4: Inferencia
### Ahora, vamos a escribir la función que usaremos en la API. Esta función toma el nombre de una película y busca en la matriz de correlación cuáles tienen el coeficiente más alto (cercano a 1.0).Paso 4: La Prueba de Fuego (Inferencia)

In [34]:
def get_recommendations(movie_title, matrix=movie_user_matrix, corr_mat=corr_matrix, top_n=10):
    try:
        # 1. Obtener el índice de la película en la matriz original
        movie_idx = list(matrix.index).index(movie_title)

        # 2. Obtener la fila de correlaciones para esa película
        corr_movie = corr_mat[movie_idx]

        # 3. Listar películas con su correlación y ordenarlas
        # Hacemos un zip de los títulos con sus correlaciones
        recommendations = list(matrix.index)
        rec_list = list(zip(recommendations, corr_movie))

        # Ordenamos de mayor a menor correlación
        rec_list.sort(key=lambda x: x[1], reverse=True)

        # 4. Devolver el top N (excluyendo la primera que es ella misma)
        return rec_list[1:top_n+1]

    except ValueError:
        return "Película no encontrada en la base de datos."

# Probamos el sistema de recomendación

In [35]:
get_recommendations("Toy Story (1995)")

[('Home Alone (1990)', np.float64(0.9446037197369745)),
 ('Jurassic Park (1993)', np.float64(0.9398803963753793)),
 ('Independence Day (a.k.a. ID4) (1996)', np.float64(0.9322034256566625)),
 ('Forrest Gump (1994)', np.float64(0.9269794061843405)),
 ('Back to the Future (1985)', np.float64(0.9258758982606918)),
 ('Willy Wonka & the Chocolate Factory (1971)',
  np.float64(0.9211604729144135)),
 ('Aladdin (1992)', np.float64(0.9211555893519637)),
 ('Jumanji (1995)', np.float64(0.9185884126716777)),
 ('Lion King, The (1994)', np.float64(0.9145189657479285)),
 ('Groundhog Day (1993)', np.float64(0.9067596840805291))]

Para Toy Story: Te recomienda Lion King, Aladdin y Jurassic Park. El modelo entendió perfectamente el concepto "Familia/Aventura de los 90s".

In [36]:
get_recommendations("Matrix, The (1999)")

[('Saving Private Ryan (1998)', np.float64(0.9645834024502957)),
 ('Gladiator (2000)', np.float64(0.9639727973358687)),
 ('Green Mile, The (1999)', np.float64(0.9634745427403372)),
 ('Lord of the Rings: The Fellowship of the Ring, The (2001)',
  np.float64(0.962036204955867)),
 ('Sixth Sense, The (1999)', np.float64(0.9615972391134227)),
 ('Star Wars: Episode IV - A New Hope (1977)', np.float64(0.948747732287984)),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)',
  np.float64(0.9392506052467966)),
 ('Lord of the Rings: The Two Towers, The (2002)',
  np.float64(0.9387734301921823)),
 ('Star Wars: Episode VI - Return of the Jedi (1983)',
  np.float64(0.9296879871978682)),
 ('Lord of the Rings: The Return of the King, The (2003)',
  np.float64(0.9265435352677261))]

Para The Matrix: Te recomienda Lord of the Rings, Star Wars y Gladiator. Capturó el concepto "Épica/Sci-Fi/Blockbuster".

# Paso 5 Guardamos el modelo

In [38]:
import pickle

# Definir ruta donde guardaremos los modelos (asegúrate de que la carpeta exista)
model_path = '../src/model/'

# 1. Guardar la lista de títulos (para mapear indices)
titles = list(movie_user_matrix.index)
with open(f'{model_path}movie_titles.pkl', 'wb') as f:
    pickle.dump(titles, f)

# 2. Guardar la matriz de correlación (el modelo en sí)
with open(f'{model_path}corr_matrix.pkl', 'wb') as f:
    pickle.dump(corr_matrix, f)

print("¡Artefactos guardados exitosamente en src/model/!")

¡Artefactos guardados exitosamente en src/model/!
